Data Validation

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import * 
import pandas as pd
import cx_Oracle as cx
from concurrent.futures import ThreadPoolExecutor
from  threading import  Thread

In [2]:
spark = SparkSession.builder.appName("Spark").getOrCreate()

In [3]:
spark

In [7]:
def validate_insert_data(lst,cnxn_pool)->dict:
    global final_dict
    validation_dict = {}
    cnxn = cnxn_pool.acquire()
    cursor = cnxn.cursor()
    qry = f"""select utc.TABLE_NAME,utc.COLUMN_NAME,utc.DATA_TYPE,utc.DATA_LENGTH,utc.NULLABLE,case when ucc.COLUMN_NAME is null then 0 else 1 end as constraint from user_tab_cols utc
    left join USER_CONS_COLUMNS ucc on (utc.TABLE_NAME = ucc.TABLE_NAME and utc.COLUMN_NAME = ucc.COLUMN_NAME) where utc.table_name IN ({','.join([f"'{i}'" for i in lst])})"""
    cursor.execute(qry)
    data = cursor.fetchall()
    # for row in data:
    #     try:
    #         validation_dict[row[0]][row[1]] = row[2:]
    #     except:
    #         validation_dict[row[0]] = {}
    #         validation_dict[row[0]][row[1]] = row[2:]
    print(data)
    cnxn_pool.release(cnxn)
    
if __name__ == '__main__':
    df_sp = spark.read.option('multiline','true').json("jsons/latestCases.json")
    cnxn_pool = cx.SessionPool('PVD_MART_30AUG','rxlogix','10.100.22.99:1521/PVSDEVDB',min=1,max=1,getmode=cx.SPOOL_ATTRVAL_WAIT,encoding='UTF-8')
    final_dict = {}
    lst = df_sp.schema.names
    lst.remove('TENANT_ID')
    lst.remove('CASE_ID')
    lst.remove('VERSION_NUM')
    validate_insert_data(lst,cnxn_pool)
    cnxn_pool.close()

[('C_AE_IDENTIFICATION', 'EVENT_RECEIPT_DATE', 'DATE', 7, 'Y', 0), ('C_AE_IDENTIFICATION', 'IID_2', 'NUMBER', 22, 'Y', 0), ('C_AE_IDENTIFICATION', 'IID_3', 'NUMBER', 22, 'Y', 0), ('C_AE_IDENTIFICATION', 'FLAG_CAUSED_HSPTALIZED', 'NUMBER', 22, 'Y', 0), ('C_AE_IDENTIFICATION', 'DEATH_EVT_RCPT_DT', 'DATE', 7, 'Y', 0), ('C_AE_IDENTIFICATION', 'FATAL_EVT_RCPT_DT', 'DATE', 7, 'Y', 0), ('C_AE_IDENTIFICATION', 'SERIOUS_EVT_RCPT_DT', 'DATE', 7, 'Y', 0), ('C_AE_IDENTIFICATION', 'AE_INTEREST_CATEGORY_DESC', 'VARCHAR2', 4000, 'Y', 0), ('C_AE_IDENTIFICATION', 'AE_INTEREST_CATEGORY_ID', 'NUMBER', 22, 'Y', 0), ('C_AE_IDENTIFICATION', 'FLAG_AE_CAT_MANUALLY_POP', 'NUMBER', 22, 'Y', 0), ('C_AE_IDENTIFICATION', 'IMDRF_CODE', 'VARCHAR2', 80, 'Y', 0), ('C_AE_IDENTIFICATION', 'NOTES', 'VARCHAR2', 32767, 'Y', 0), ('C_AE_IDENTIFICATION', 'CD_DATE_ATTR_DELETED', 'NUMBER', 22, 'Y', 0), ('C_AE_IDENTIFICATION', 'DATE_ATTR_DELETED', 'DATE', 7, 'Y', 0), ('C_AE_IDENTIFICATION', 'SYS_NC00098$', 'RAW', 126, 'Y', 0), (

In [ ]:
lst = df_sp.schema.names
# lst

def split_file(a,n)->list:
    k,m = divmod(len(a),n)
    return list((a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)))

rt = split_file(lst,5)  

In [ ]:
r = [('C_NARRATIVE_ADDL', 'CASE_ID', 'NUMBER', 22, 'N', 1), ('C_NARRATIVE_ADDL', 'TENANT_ID', 'NUMBER', 22, 'N', 1), ('C_NARRATIVE_ADDL', 'VERSION_NUM', 'NUMBER', 22, 'N', 1), ('C_NARRATIVE_ADDL', 'DELETED_FLAG', 'NUMBER', 22, 'Y', 0), ('C_NARRATIVE_ADDL', 'IID_1', 'NUMBER', 22, 'Y', 0), ('C_NARRATIVE_ADDL', 'TXT_DATE_ATTR_DELETED', 'VARCHAR2', 200, 'Y', 0), ('C_NARRATIVE_ADDL', 'DELETED', 'DATE', 7, 'Y', 0), ('C_NARRATIVE_ADDL', 'CD_DATE_ATTR_DELETED', 'NUMBER', 22, 'Y', 0), ('C_NARRATIVE_ADDL', 'DATE_VERSION_END', 'DATE', 7, 'Y', 0), ('C_NARRATIVE_ADDL', 'FLAG_DELETED', 'NUMBER', 22, 'Y', 0), ('C_NARRATIVE_ADDL', 'SYS_NC00012$', 'RAW', 126, 'Y', 0), ('C_NARRATIVE_ADDL', 'ABBREV_NARRATIVE', 'CLOB', 4000, 'Y', 0), ('C_NARRATIVE_ADDL', 'DATE_VERSION_START', 'DATE', 7, 'Y', 0), ('C_NARRATIVE_ADDL', 'DATE_ATTR_DELETED', 'DATE', 7, 'Y', 0)]
dict = {}

for i in r:
    try:
        dict[i[0]][i[1]] = i[2:]
    except:
        dict[i[0]] = {}
        dict[i[0]][i[1]] = i[2:]
        
print(dict)


